In [1]:
"""
Este script define o processo completo de treinamento e avaliação de um modelo de análise de sentimentos baseado em LSTM.
Ele envolve o carregamento e pré-processamento de dados, criação de datasets, configuração e treinamento do modelo,
avaliação do desempenho em um conjunto de validação e teste, e, finalmente, salva o modelo treinado e os artefatos necessários.

Dependências:
- PyTorch: Utilizado para construir e treinar a rede neural.
- Scikit-Learn: Usado para métricas de avaliação e pré-processamento.
- Tabulate: Para formatar tabelas de resultados de classificação.
- TQDM: Fornece uma barra de progresso para loops.
- Pickle: Para salvar o modelo e o vocabulário.

Funções Principais:
- `collate_batch`: Prepara batches de dados, gerenciando padding e conversão para tensores PyTorch.
- `print_classification_report`: Imprime um relatório formatado das métricas de classificação.
- `evaluate_model`: Avalia o modelo no conjunto de dados fornecido.
- `train_and_evaluate`: Treina o modelo usando dados de treinamento e valida usando um conjunto de validação.
- `main`: Orquestra o processo de carregamento de dados, treinamento, avaliação e salvamento de artefatos.

Fluxo:
1. Dados são carregados e pré-processados.
2. Datasets para treino, validação e teste são criados.
3. Um modelo LSTM é instanciado e treinado.
4. O modelo é avaliado em dados de validação e teste.
5. Resultados são impressos e o modelo é salvo.

Membros do grupo:

Alonso Batista de Oliveira Júnior
André Moreira de Carvalho
Gustavo Castro Candeia
Halex Maciel Silva Vieira
Welbert Luiz Silva Junior


Observação:
    
    Treinando com learning rate de 0.0001
    
"""

import os
import warnings
from copy import deepcopy
from typing import Tuple

import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from tabulate import tabulate
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import SentimentLSTM
from utils import create_datasets, load_and_preprocess_data, tokenize_and_encode, CONFIG
import pickle

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\halex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def collate_batch(batch):
    """
    Função para preparar batches de dados, aplicando padding e convertendo listas de textos e rótulos para tensores PyTorch.
    
    Args:
        batch (list of tuples): Uma lista de tuplas, onde cada tupla contém um tensor de texto e seu respectivo rótulo.
    
    Returns:
        Tuple[torch.Tensor, torch.Tensor, torch.Tensor]: Tensores de textos, rótulos e comprimentos originais dos textos.
    """
    texts, labels = zip(*[(item[0], item[1]) for item in batch])
    labels = torch.tensor(labels, dtype=torch.long)
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=0)  # Assume-se que 0 é o índice de padding
    lengths = torch.tensor([len(text) for text in texts], dtype=torch.long)
    return texts, labels, lengths

def print_classification_report(report: dict) -> None:
    """
    Imprime um relatório formatado das métricas de classificação usando a biblioteca tabulate.
    
    Args:
        report (dict): Um dicionário contendo as métricas de classificação para cada classe.
    """
    headers = ["Classe", "Precisão", "Revocação", "Pontuação F1"]
    rows = []
    for label in ['negative', 'positive', 'uncertainty', 'litigious']:
        rows.append([
            label.capitalize(),
            f"{report[label]['precision']:.2f}",
            f"{report[label]['recall']:.2f}",
            f"{report[label]['f1-score']:.2f}"
        ])
    print(tabulate(rows, headers=headers, tablefmt="grid"))
    print(f"\nAcurácia Geral: {report['accuracy']:.2f}%")

In [ ]:
def evaluate_model(model: nn.Module, loader: DataLoader, device: torch.device) -> Tuple[float, float, dict]:
    """
    Avalia o modelo no conjunto de dados fornecido calculando perda, acurácia e outras métricas de classificação.
    
    Args:
        model (nn.Module): O modelo de rede neural a ser avaliado.
        loader (DataLoader): DataLoader fornecendo os dados de avaliação.
        device (torch.device): Dispositivo de hardware onde o modelo está sendo executado (CPU ou GPU).
    
    Returns:
        Tuple[float, float, dict]: Média de perdas, acurácia e um dicionário com o relatório de classificação.
    """
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    y_true = []
    y_pred = []
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for texts, labels, lengths in loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = total_correct / total_samples
    report = classification_report(y_true, y_pred, target_names=['negative', 'positive', 'uncertainty', 'litigious'], output_dict=True, zero_division=0)
    return total_loss / len(loader), accuracy, report

In [ ]:
def train_and_evaluate(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, device: torch.device) -> nn.Module:
    """
    Treina o modelo usando dados de treinamento e valida usando um conjunto de validação.
    Implementa redução de taxa de aprendizado em plateau e parada antecipada para evitar overfitting.
    
    Args:
        model (nn.Module): O modelo a ser treinado e avaliado.
        train_loader (DataLoader): DataLoader para o conjunto de treinamento.
        val_loader (DataLoader): DataLoader para o conjunto de validação.
        device (torch.device): Dispositivo onde o modelo será executado.
    
    Returns:
        nn.Module: O modelo com os pesos ajustados pelo treinamento.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=CONFIG['lr'], weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    best_val_loss = float('inf')
    best_model = None
    no_improve_epochs = 0
    early_stopping_patience = 10

    for epoch in range(CONFIG['num_epochs']):
        model.train()
        train_loss = 0
        train_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{CONFIG['num_epochs']} - Training")

        for texts, labels, lengths in train_iter:
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_iter.set_postfix(loss=loss.item())

        val_loss, val_accuracy, val_report = evaluate_model(model, val_loader, device)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model.state_dict())
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1

        print(f"Época {epoch+1}: Perda no Treino: {train_loss / len(train_loader)}, Perda na Validação: {val_loss}")
        print(f"Acurácia na validação: {val_accuracy:.2f}%")
        print("Relatório de Classificação da Validação:")
        print_classification_report(val_report)

        if no_improve_epochs >= early_stopping_patience:
            print(f"Parada antecipada após {early_stopping_patience} épocas sem melhora.")
            break

    model.load_state_dict(best_model)
    return model

In [ ]:
def main():
    """
    Função principal que orquestra o processo de carregamento dos dados, criação dos datasets,
    inicialização do modelo, treinamento, avaliação e salvamento dos artefatos do modelo.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device.type.upper()}, {torch.cuda.get_device_name(0) if device.type == 'cuda' else 'CPU'}")

    print("Carregando e processando os dados...")
    df, label_encoder, vocab = load_and_preprocess_data(CONFIG['data_dir'])
    print("Dados carregados e processados com sucesso.")

    print("Criando datasets...")
    train_dataset, val_dataset, test_dataset = create_datasets(df, label_encoder, vocab, CONFIG)
    print("Datasets criados com sucesso.")

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], collate_fn=collate_batch)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], collate_fn=collate_batch)

    print("Inicializando o modelo LSTM...")
    model = SentimentLSTM(len(vocab), CONFIG).to(device)
    print(f"Modelo inicializado e movido para {device.type}")

    print("Iniciando o processo de treinamento...")
    trained_model = train_and_evaluate(model, train_loader, val_loader, device)
    print("Treinamento concluído com sucesso.")

    print("Avaliando o modelo nos dados de teste...")
    test_loss, test_accuracy, test_report = evaluate_model(trained_model, test_loader, device)
    print(f"Perda em teste: {test_loss:.4f}")
    print(f"Acurácia em teste: {test_accuracy:.2f}%")
    print("Relatório de classificação:\n")
    print_classification_report(test_report)
    print("Avaliação em teste concluída.")

    print("Salvando o modelo treinado e vocabulário...")
    if not os.path.exists(CONFIG['models_dir']):
        os.makedirs(CONFIG['models_dir'])
    torch.save(trained_model.state_dict(), os.path.join(CONFIG['models_dir'], 'sentiment_lstm_model_v1.pth'))
    with open(os.path.join(CONFIG['models_dir'], 'vocab_v1.pkl'), 'wb') as f:
        pickle.dump(vocab, f)
    with open(os.path.join(CONFIG['models_dir'], 'label_encoder_v1.pkl'), 'wb') as f:
        pickle.dump(label_encoder, f)
    print("Modelo, vocabulário, e codificador de label salvos com sucesso.")

In [ ]:
# Treinando com learning rate de 0.0001

if __name__ == '__main__':
    main()

Usando dispositivo: CUDA, NVIDIA GeForce RTX 3060
Carregando e processando os dados...
Mapeamento dos Labels: {'litigious': 0, 'negative': 1, 'positive': 2, 'uncertainty': 3}
Dados carregados e processados com sucesso.
Criando datasets...


Tokenizando Textos: 100%|██████████| 131352/131352 [00:24<00:00, 5287.50it/s]


Datasets criados com sucesso.
Inicializando o modelo LSTM...
Modelo inicializado e movido para cuda
Iniciando o processo de treinamento...


Epoch 1/50 - Training: 100%|██████████| 6158/6158 [08:52<00:00, 11.56it/s, loss=0.00784]


Época 1: Perda no Treino: 0.42174238221660576, Perda na Validação: 0.11672683508864377
Acurácia na validação: 0.96%
Relatório de Classificação da Validação:
+-------------+------------+-------------+----------------+
| Classe      |   Precisão |   Revocação |   Pontuação F1 |
+=============+============+=============+================+
| Negative    |       0.93 |        0.99 |           0.96 |
+-------------+------------+-------------+----------------+
| Positive    |       0.98 |        0.93 |           0.96 |
+-------------+------------+-------------+----------------+
| Uncertainty |       0.97 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+
| Litigious   |       0.95 |        0.96 |           0.95 |
+-------------+------------+-------------+----------------+

Acurácia Geral: 0.96%


...

Época 18: Perda no Treino: 0.04416934269976165, Perda na Validação: 0.11084862965369154
Acurácia na validação: 0.96%
Relatório de Classificação da Validação:
+-------------+------------+-------------+----------------+
| Classe      |   Precisão |   Revocação |   Pontuação F1 |
+=============+============+=============+================+
| Negative    |       0.97 |        0.98 |           0.97 |
+-------------+------------+-------------+----------------+
| Positive    |       0.96 |        0.97 |           0.96 |
+-------------+------------+-------------+----------------+
| Uncertainty |       0.97 |        0.96 |           0.97 |
+-------------+------------+-------------+----------------+
| Litigious   |       0.95 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+

Acurácia Geral: 0.96%


Epoch 19/50 - Training: 100%|██████████| 6158/6158 [09:07<00:00, 11.26it/s, loss=0.000518]


Época 19: Perda no Treino: 0.04138794464003632, Perda na Validação: 0.11768892919304484
Acurácia na validação: 0.96%
Relatório de Classificação da Validação:
+-------------+------------+-------------+----------------+
| Classe      |   Precisão |   Revocação |   Pontuação F1 |
+=============+============+=============+================+
| Negative    |       0.97 |        0.97 |           0.97 |
+-------------+------------+-------------+----------------+
| Positive    |       0.96 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+
| Uncertainty |       0.97 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+
| Litigious   |       0.95 |        0.96 |           0.95 |
+-------------+------------+-------------+----------------+

Acurácia Geral: 0.96%


Epoch 20/50 - Training: 100%|██████████| 6158/6158 [08:55<00:00, 11.50it/s, loss=0.0131]  


Época 20: Perda no Treino: 0.03892450783981989, Perda na Validação: 0.11519571518405493
Acurácia na validação: 0.96%
Relatório de Classificação da Validação:
+-------------+------------+-------------+----------------+
| Classe      |   Precisão |   Revocação |   Pontuação F1 |
+=============+============+=============+================+
| Negative    |       0.98 |        0.97 |           0.97 |
+-------------+------------+-------------+----------------+
| Positive    |       0.96 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+
| Uncertainty |       0.97 |        0.96 |           0.97 |
+-------------+------------+-------------+----------------+
| Litigious   |       0.95 |        0.96 |           0.96 |
+-------------+------------+-------------+----------------+

Acurácia Geral: 0.96%
Parada antecipada após 10 épocas sem melhora.
Treinamento concluído com sucesso.
Avaliando o modelo nos dados de teste...
Perda em teste: 0.0861
Acurácia em te